In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load dataset
df = pd.read_csv('/kaggle/input/churn-prediction-dataset/train.csv')

Confidence interval (95%)
-

In [ ]:
# Select category variables
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f"\nNumeric variables ({len(numeric_cols)}):\n", numeric_cols)

# Function to calculate category variables
def confidence_interval(series, confidence=0.95):
    data = series.dropna()
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    se = std / np.sqrt(n)
    alpha = 1 - confidence
    t_crit = stats.t.ppf(1 - alpha/2, df=n-1)
    margin = t_crit * se
    lower = mean - margin
    upper = mean + margin

    return pd.Series({
        "Sample Size (n)": n,
        "Mean (x̄)": mean,
        "Std Dev (s)": std,
        "Std Error": se,
        "t-critical": t_crit,
        "Margin of Error": margin,
        "95% CI Lower": lower,
        "95% CI Upper": upper
    })

# Apply for all var
ci_results = df[numeric_cols].apply(confidence_interval).T

# Display the result
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
display(ci_results)


 Use Independent two-sample t-test to find the relationship between 2 groups churn and non-churn.
 -
 Compare and display statisticable results.

In [ ]:
# Select the numeric variables
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# If it has a target var'churn', remove
if 'Churn' in numeric_cols:
    numeric_cols.remove('Churn')

print(f"Numeric variables ({len(numeric_cols)}): {numeric_cols}")
print(f"Categorical variables ({len(categorical_cols)}): {categorical_cols}")

# Devide into 2 groups (Churned vs Non-Churned)
churned = df[df['Churn'] == 1]
non_churned = df[df['Churn'] == 0]

# Independent T-test 
def t_test_by_group(var):
    x1 = churned[var].dropna()
    x2 = non_churned[var].dropna()

    # Levene's test for 2 variances equal or not
    _, p_levene = stats.levene(x1, x2)
    equal_var = p_levene > 0.05  # True nếu phương sai không khác biệt

    
    # t-test for 2 independent samples
    t_stat, p_value = stats.ttest_ind(x1, x2, equal_var=equal_var)

    return pd.Series({
        'Mean_Churned': x1.mean(),
        'Mean_NonChurned': x2.mean(),
        'Difference': x1.mean() - x2.mean(),
        't-statistic': t_stat,
        'p-value': p_value,
        'Equal Variance?': equal_var,
        'Significant? (α=0.05)': 'Yes' if p_value < 0.05 else 'No'
    })


# Apply for all var
t_test_results = pd.DataFrame({var: t_test_by_group(var) for var in numeric_cols}).T

# Chi-square
chi2_results = {}

for col in categorical_cols:
    try:
        ct = pd.crosstab(df[col], df['Churn'])
        chi2, p, dof, ex = stats.chi2_contingency(ct)
        chi2_results[col] = {'Chi2 Statistic': chi2, 'p-value': p, 'DoF': dof, 
                             'Significant? (α=0.05)': 'Yes' if p < 0.05 else 'No'}
    except Exception as e:
        chi2_results[col] = {'Chi2 Statistic': np.nan, 'p-value': np.nan, 'DoF': np.nan, 'Significant? (α=0.05)': 'Error'}

chi2_df = pd.DataFrame(chi2_results).T

# Display the results
pd.set_option('display.float_format', lambda x: f"{x:,.4f}")
display(t_test_results.sort_values('p-value'))
display(chi2_df.sort_values('p-value'))

p-value < 0.05 → Bác bỏ H₀ → Biến có sự khác biệt có ý nghĩa thống kê giữa nhóm churn và non-churn.

p-value ≥ 0.05 → Không bác bỏ H₀ → Không có sự khác biệt rõ rệt.

In [ ]:
# Create bõ plots for meaningful variables (p < 0.05)
significant_vars = t_test_results[t_test_results['Significant? (α=0.05)'] == 'Yes'].index.tolist()

print(f"\nBiến có ý nghĩa thống kê (p < 0.05): {significant_vars}")

for var in significant_vars[:5]:  # chỉ vẽ 5 biến đầu tiên
    plt.figure(figsize=(6,4))
    sns.boxplot(x='Churn', y=var, data=df, palette='Set2')
    plt.title(f"Distribution of {var} by Churn Status (p < 0.05)")
    plt.show()

> Levene's test cho tất cả các phương sai của biến category

> Chi-square test cho biến category

In [ ]:
# Chi-square Test: kiểm tra mối quan hệ giữa Churn và các biến phân loại
cat_cols = df.select_dtypes(include='object').columns.tolist()
chi2_results = {}

for col in cat_cols:
    ct = pd.crosstab(df[col], df['Churn'])
    chi2, p, dof, ex = stats.chi2_contingency(ct)
    chi2_results[col] = {'Chi2 Statistic': chi2, 'p-value': p, 'DoF': dof}

chi2_df = pd.DataFrame(chi2_results).T
display(chi2_df.sort_values('p-value'))


ADVANCED ANOVA PIPELINE (ONE-WAY, TWO-WAY, POST-HOC)
-

In [ ]:
# ADVANCED ANOVA PIPELINE (ONE-WAY, TWO-WAY, POST-HOC)

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats

# PHÂN LOẠI BIẾN
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Nếu biến mục tiêu 'Churn' là numeric, ta bỏ nó ra
if 'Churn' in numeric_cols:
    numeric_cols.remove('Churn')

# ONE-WAY ANOVA CHO TỪNG CẶP (num × cat)
anova_results = []

for num in numeric_cols:
    for cat in categorical_cols:
        try:
            groups = [df[df[cat] == c][num].dropna() for c in df[cat].dropna().unique()]
            if len(groups) >= 2:  # chỉ xét nếu có ít nhất 2 nhóm
                f_stat, p_val = stats.f_oneway(*groups)
                anova_results.append({
                    'Numeric_Var': num,
                    'Categorical_Var': cat,
                    'F-statistic': f_stat,
                    'p-value': p_val
                })
        except Exception as e:
            pass

anova_df = pd.DataFrame(anova_results)
anova_df['Significant?'] = np.where(anova_df['p-value'] < 0.05, 'Yes', 'No')

print("✅ ONE-WAY ANOVA Results:")
display(anova_df.sort_values('p-value').reset_index(drop=True))


# TWO-WAY ANOVA (kết hợp 2 biến phân loại ảnh hưởng đến 1 biến số)
two_way_results = []

for num in numeric_cols:
    for i in range(len(categorical_cols)):
        for j in range(i + 1, len(categorical_cols)):
            cat1, cat2 = categorical_cols[i], categorical_cols[j]
            try:
                model = ols(f'{num} ~ C({cat1}) * C({cat2})', data=df).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)
                p_main1 = anova_table.loc[f'C({cat1})', 'PR(>F)']
                p_main2 = anova_table.loc[f'C({cat2})', 'PR(>F)']
                p_interaction = anova_table.loc[f'C({cat1}):C({cat2})', 'PR(>F)']
                two_way_results.append({
                    'Numeric_Var': num,
                    'Cat1': cat1,
                    'Cat2': cat2,
                    'p_Cat1': p_main1,
                    'p_Cat2': p_main2,
                    'p_Interaction': p_interaction
                })
            except Exception as e:
                pass

two_way_df = pd.DataFrame(two_way_results)
print("\n✅ TWO-WAY ANOVA Results:")
display(two_way_df.sort_values('p_Interaction').reset_index(drop=True))


# 4️⃣ POST-HOC TEST (Tukey HSD) cho các cặp biến có ý nghĩa
posthoc_results = []

for idx, row in anova_df.iterrows():
    if row['p-value'] < 0.05:  # chỉ chạy post-hoc nếu ANOVA có ý nghĩa
        num = row['Numeric_Var']
        cat = row['Categorical_Var']
        try:
            tukey = pairwise_tukeyhsd(endog=df[num], groups=df[cat], alpha=0.05)
            tukey_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
            tukey_df['Numeric_Var'] = num
            tukey_df['Categorical_Var'] = cat
            posthoc_results.append(tukey_df)
        except Exception as e:
            pass

if posthoc_results:
    posthoc_df = pd.concat(posthoc_results, ignore_index=True)
    print("\n POST-HOC (Tukey HSD) Results for Significant ANOVAs:")
    display(posthoc_df)
else:
    print("\n No significant ANOVA results found for post-hoc testing.")
